In [ ]:
from model_manager import section1, section2, single_model
import optimizer as opt
import dataset_creator
from constants import *

# Datensatz 1 erstellung
## Grösse kann angepasst werden: Max Grösse 60000, für train_size und 10000 für test_size

In [ ]:
dataset1_generator = dataset_creator.Dataset1()
dataset1_generator.generate_dataset(train_size=10000, test_size=5000, whole_dataset=False)

import gc
del dataset1_generator
gc.collect()

# Koordinatennetz optimieren
## Alle Werte sind anpassbar
## max_trials sind Maximale Anzahl Optimierungsschritten, target ist der Ziel-MAE

In [ ]:
optim = opt.BayesianOptimizer()


params = {
    'min_conv_layers': 1,
    'max_conv_layers': 5,
    'min_dense_layers': 1,
    'max_dense_layers': 5,
    'min_conv_size': 4,
    'max_conv_size': 256,
    'min_dense_size': 4,
    'max_dense_size': 256,
    'epochs': 50,
    'flatten_type': ['global_average'],
    'activation': ['relu'],
    'optimizer': ['adam'],
    'learning_rate': [0.001],
    'dropout': [0.01],
}


model = section1()

best_trial = optim.optimize(model=model, max_trials=3, target=5, initial_params=params, write_to_file=True, penalty_blob='multiply')

# Datensatz 2 erstellung
## einfach ausführen
## nur möglich, falls ein Koordinatennetz schon trainiert ist, mit trials.json file im Optimizationsordner

In [ ]:
# dieser Teil wurde mit chatGPT generiert
import json

def load_params(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    return data

def transform_params(params):
    best_conv = [param for key, param in params.items() if key.startswith('conv')]
    best_dense = [param for key, param in params.items() if key.startswith('dense')]

    params['conv_layers'] = best_conv
    params['dense_layers'] = best_dense
    return params

def params_generator(data):
    for entry in data:
        yield (transform_params(entry["params"]), entry["trial_number"])

def update_mae(params, n):
    model = section1()
    model.initialise_data_and_model(train_params=params, weights=SECT1_CHECKPOINT_FOLDER + f'regression_model_optimize_{n}.weights.h5')
    model.model.load_weights(SINGLE_CHECKPOINT_FOLDER + f'single_model_optimize_{n}.weights.h5')
    train_params = {
        "epochs": 1
    }
    _,_,_,history =model.train(params=train_params)

    mae = history['val_mean_absolute_error'][0]
    crop_amount = 32
    grace = int(round(mae/2, 0))
    
    change_cropped_image_shape([crop_amount+2*grace,crop_amount+2*grace,1])
    CROPPED_IMAGE = {
        'image_shape': [crop_amount+2*grace,crop_amount+2*grace,1],
        'input_shape': tuple([crop_amount+2*grace,crop_amount+2*grace,1])
    }
    print(CROPPED_IMAGE)

if __name__ == "__main__":
    file_path = OPTIMIZER_FOLDER + "trials.json"
    #file_path = "H:\\aless\\Documents\\Python_Scripts\\Matur\\matura-private-main\\matura-private-1\\resilt2\\sect1.json"

    data = load_params(file_path)

    generator = params_generator(data)

    best_time = float("inf")
    best_iter = 0
    best_params = {}
    for i, (params, n) in enumerate(generator):
        print("Next Parameters:", params, "trial number", n)

        #update_mae(params, n)

        #dataset2_gen = dataset_creator.Dataset2(weights="H:\\aless\\Documents\\Python_Scripts\\Matur\\matura-private-main\\matura-private-1\\resilt2\\best_sect1.weights.h5", trainable_params = params)
        dataset2_gen = dataset_creator.Dataset2(weights=SECT1_CHECKPOINT_FOLDER + f'regression_model_optimize_{n}.weights.h5', trainable_params = params)
        dataset2_gen.generate_dataset()


        break

# Klassifikationsnetz optimieren
## Alle Werte sind anpassbar
## max_trials sind Maximale Anzahl Optimierungsschritten, target ist die Ziel-Accuracy

In [ ]:
optim = opt.BayesianOptimizer()


params = {
    'min_conv_layers': 1,
    'max_conv_layers': 3,
    'min_dense_layers': 1,
    'max_dense_layers': 3,
    'min_conv_size': 4,
    'max_conv_size': 128,
    'min_dense_size': 4,
    'max_dense_size': 128,
    'epochs': 50,
    'flatten_type': ['global_average'],
    'activation': ['relu'],
    'optimizer': ['adam'],
    'learning_rate': [0.001],
    'dropout': [0.01],
}


model = section2()

best_trial = optim.optimize(model=model, max_trials=3, target=0.7, initial_params=params, write_to_file=True, penalty_blob='divide')

# SingleNet optimieren
## Alle Werte sind anpassbar
## max_trials sind Maximale Anzahl Optimierungsschritten, target ist die Ziel-accuracy

In [ ]:
optim = opt.BayesianOptimizer()


params = {
    'min_conv_layers': 1,
    'max_conv_layers': 5,
    'min_dense_layers': 1,
    'max_dense_layers': 5,
    'min_conv_size': 4,
    'max_conv_size': 256,
    'min_dense_size': 4,
    'max_dense_size': 256,
    'epochs': 50,
    'flatten_type': ['global_average'],
    'activation': ['relu'],
    'optimizer': ['adam'],
    'learning_rate': [0.001],
    'dropout': [0.01],
}


model = single_model()

best_trial = optim.optimize(model=model, max_trials=3, target=0.7, initial_params=params, write_to_file=True, penalty_blob='divide')